In [148]:
"""
Statistics: count difference between anti-deuteron and anti-proton
@aurthor Jiancheng Zeng
@July 15 2021

"""

In [81]:
import numpy as np
import matplotlib.pyplot as plt
import statistics
from numpy import array
import pandas as pd

event = []
identical = []
En = 40

"""
Select by name of the column
---------------------
"""
title = ["eventID", "trackID", "parentID", "particleID", "copyNb1 (mother volume)", "copyNb", "material", "time (ns)", "energy(MeV)", "energy deposit(MeV)", "parent process name", "process name", "px (MeV/c)", "py (MeV/c)", "pz (MeV/c)", "step length (cm)", "x (cm)", "y (cm)", "z (cm)"]

"""
Read data from total data file
--------------------
"""
print("Reading data ...")
data = pd.read_table("/Users/jiancheng/GRAMS/Toy Model/Machine Learning/210822_anti_proton_40MeV.dat")    #get data file
print("finish reading data!")

"""
modification
"""
print("Modifing data ...")
for i in range(0, len(data.iloc[:,0])):
    if(0 <= data.iloc[i,2] <= 1):
        event.append(list(data.iloc[i,:]))
    print("Modifing process:"+str(int(100*i/len(data.iloc[:,0])))+"%", end='\r')
event = np.array(event)


"""
recognize the data file length distribution
"""
print("\n Recognize events ...")
s = 0
temp = 1;
while s < len(event[:,0]):
    if(int(event[s,0]) == temp):
        identical.append(s)
        temp = temp + 1
    s = s + 1
    print("Recognizing process:"+str(int(100*s/len(event[:,0])))+"%", end='\r')
    #time.sleep(0.001)
selection = np.split(event, identical)

identical.append(len(event))

np.savetxt('/Users/jiancheng/GRAMS/Toy Model/Machine Learning/210822_anti_proton_40MeV_modified.txt', selection, fmt='%s')

print("finished!")

Reading data ...
finish reading data!
Modifing data ...
Modifing process:99%
 Recognize events ...
finished!ng process:100%


In [88]:
PeD = [None] * len(identical)    #Penetration depth
EnD = [None] * len(identical)    #Energy deposit
NuPip = [0] * len(identical)     #Number of pion+
NuPim = [0] * len(identical)     #Number of pion-
NuPi0 = [0] * len(identical)     #Number of neutral pion
NuP = [0] * len(identical)       #Number of proton
XRay = [0] * len(identical)      #X-ray
lDetector = 2000                 #detector depth
temp = 0
step = [0]
step.extend(identical)
print(len(step))
print(identical)

"""
find penatration depth, number of pions and X-ray
"""
for i in range(len(identical)):
    for j in range(step[i+1] - step[i]):
        if(event[step[i] + j][3] != '-2212'):
            print(event[temp + j - 1][18])
            PeD[i] = lDetector - float(event[temp + j - 1][18])
            EnD[i] = En - float(event[temp + j - 2][8])
            break
    for k in range(step[i+1] - step[i] - 1):
        if(event[temp + k][1] != event[temp + k + 1][1]):
            if(event[temp + k][3] == '211'):
                NuPip[i] = NuPip[i] + 1
            elif(event[temp + k][3] == '-211'):
                NuPim[i] = NuPim[i] + 1
            elif(event[temp + k][3] == '111'):
                NuPi0[i] = NuPi0[i] + 1
            elif(event[temp + k][3] == '2212'):
                NuP[i] = NuP[i] + 1
    if(event[identical[i] - 1][3] == '211'):    #specify last track
        NuPip[i] = NuPip[i] + 1
    elif(event[identical[i] - 1][3] == '-211'):
        NuPim[i] = NuPim[i] + 1
    elif(event[iidentical[i] - 1][3] == '111'):
        NuPi0[i] = NuPi0[i] + 1
    elif(event[identical[i] - 1][3] == '2212'):
        NuP[i] = NuP[i] + 1
    temp = identical[i]
    print("forming vectors...:"+str(int(100*i/len(identical)))+"%", end='\r')
    

"""
print("penetration depth:")
print(PeD)
print("\n number of pi+:")
print(NuPip)
print("\n number of pi-:")
print(NuPim)
print("\n number of pi0:")
print(NuPi0)
print("\n number of proton:")
print(NuP)
"""

vector=[PeD, EnD, NuPip, NuPim, NuPi0, NuP]    #vector content follow by: Penetration depth, energy diposit, number of positive pions, number of negative pions, number of neutral pions, number of protons
vector=np.transpose(vector)

np.savetxt('/Users/jiancheng/GRAMS/Toy Model/Machine Learning/210822_anti_proton_40MeV_vector.txt', vector, delimiter = '\t ', fmt='%.6s')
print("\n Done! Vector saved!")


501
[39, 110, 252, 343, 479, 555, 594, 736, 803, 925, 1023, 1073, 1165, 1194, 1237, 1310, 1362, 1422, 1514, 1627, 1676, 1706, 1770, 1854, 1935, 2005, 2100, 2150, 2197, 2257, 2385, 2471, 2583, 2630, 2674, 2714, 2847, 2924, 3062, 3120, 3172, 3210, 3317, 3396, 3488, 3634, 3695, 3740, 3838, 3969, 4125, 4168, 4231, 4309, 4398, 4470, 4573, 4642, 4690, 4729, 4789, 4843, 4867, 4940, 5015, 5123, 5238, 5330, 5387, 5433, 5528, 5616, 5712, 5755, 5805, 5884, 5915, 6019, 6050, 6079, 6191, 6304, 6342, 6384, 6427, 6472, 6545, 6637, 6767, 6820, 6918, 7019, 7074, 7134, 7236, 7340, 7482, 7611, 7645, 7695, 7735, 7870, 7944, 8041, 8138, 8218, 8306, 8403, 8502, 8643, 8764, 8844, 8985, 9086, 9190, 9242, 9330, 9393, 9435, 9524, 9584, 9670, 9714, 9779, 9863, 9975, 10032, 10143, 10202, 10230, 10337, 10391, 10440, 10561, 10607, 10690, 10764, 10884, 10919, 11013, 11055, 11096, 11112, 11220, 11266, 11417, 11475, 11606, 11741, 11812, 11902, 11945, 12046, 12125, 12227, 12293, 12369, 12439, 12503, 12558, 12649, 12689

In [77]:
print(identical)

[39, 110, 252, 343, 479, 555, 594, 736, 803, 925, 1023, 1073, 1165, 1194, 1237, 1310, 1362, 1422, 1514, 1627, 1676, 1706, 1770, 1854, 1935, 2005, 2100, 2150, 2197, 2257, 2385, 2471, 2583, 2630, 2674, 2714, 2847, 2924, 3062, 3120, 3172, 3210, 3317, 3396, 3488, 3634, 3695, 3740, 3838, 3969, 4125, 4168, 4231, 4309, 4398, 4470, 4573, 4642, 4690, 4729, 4789, 4843, 4867, 4940, 5015, 5123, 5238, 5330, 5387, 5433, 5528, 5616, 5712, 5755, 5805, 5884, 5915, 6019, 6050, 6079, 6191, 6304, 6342, 6384, 6427, 6472, 6545, 6637, 6767, 6820, 6918, 7019, 7074, 7134, 7236, 7340, 7482, 7611, 7645, 7695, 7735, 7870, 7944, 8041, 8138, 8218, 8306, 8403, 8502, 8643, 8764, 8844, 8985, 9086, 9190, 9242, 9330, 9393, 9435, 9524, 9584, 9670, 9714, 9779, 9863, 9975, 10032, 10143, 10202, 10230, 10337, 10391, 10440, 10561, 10607, 10690, 10764, 10884, 10919, 11013, 11055, 11096, 11112, 11220, 11266, 11417, 11475, 11606, 11741, 11812, 11902, 11945, 12046, 12125, 12227, 12293, 12369, 12439, 12503, 12558, 12649, 12689, 12